# Evaluation of CatVAE discretization performance with preprocessed tank Dataset
Visual evaluation of the preciseness of discretization and meaningful categories. <br>
Wihtin the first subplot we plot a selection of measurement values from the dataset. <br>
The second subplot includes the learned discretizations from the CatVAE indicated by the index for the x-axis. <br>
The third subplot plots the loglikelihood of the CatVAE according to the the input data. <br>

To make a statement about the change in states and likelihood, we plot one codeblock with nominal system behavior and the following with anomalous system behavior. 

In [ ]:
cd ..

In [ ]:
import os
import pandas as pd
import torch
import plotly.graph_objects as go
import yaml
from plotly.subplots import make_subplots
import numpy as np
from ipywidgets import interact

from utils import standardize_data, preprocess_data_simu_tank, preprocess_std_data
# from datamodule import Dataset
from seqcat_datamodule import Dataset

from catvae_tank import CategoricalVAE
from seqcat_catvae import seqcat_vae
from seq_vae import seq_vae
from seq_datamodule import Dataset as seq_dataset
from sklearn.preprocessing import StandardScaler
np.random.seed(123)
torch.manual_seed(123)


In [ ]:
def plot_like(idx, anomaly):

    # MODEL_VERSION = f'CatVAE_training_hparams/tank_simulation'
    # ckpt_file_name = os.listdir(f'./{MODEL_VERSION}/checkpoints/')[-1]
    # ckpt_file_path = f'./{MODEL_VERSION}/checkpoints/{ckpt_file_name}'
    # with open(f'./{MODEL_VERSION}/hparams.yaml') as f:
    #     hparam = yaml.safe_load(f)
    # model = CategoricalVAE(hparams=hparam["hparams"]).load_from_checkpoint(ckpt_file_path)

    # MODEL_VERSION = f'lightning_logs/seqcat_tank_noRNN_v11/version_10'
    # MODEL_VERSION = f'lightning_logs/seqcat_tank_RNN_v17/version_32' # version 3 and version 25
    # MODEL_VERSION = f'lightning_logs/seqcat_tank_long_v18/version_21' # version 3 and version 25
    MODEL_VERSION = f'lightning_logs/seqcat_tank_RNN_v25/version_5' # version 3 and version 25


    ckpt_file_name = os.listdir(f'./{MODEL_VERSION}/checkpoints/')[-1]
    ckpt_file_path = f'./{MODEL_VERSION}/checkpoints/{ckpt_file_name}'
    with open(f'./{MODEL_VERSION}/hparams.yaml') as f:
        hparam = yaml.safe_load(f)
    model = seqcat_vae.load_from_checkpoint(ckpt_file_path, hparams=hparam["hparams"])

    MODEL_VERSION_SEQ = f'lightning_logs/seq_vae_tank_v9/version_5' # 
    threshold = -170
    ckpt_file_name_seq = os.listdir(f'./{MODEL_VERSION_SEQ}/checkpoints/')[-1]
    ckpt_file_path_seq = f'./{MODEL_VERSION_SEQ}/checkpoints/{ckpt_file_name_seq}'
    with open(f'./{MODEL_VERSION_SEQ}/hparams.yaml') as f:
        hparam_seq = yaml.safe_load(f)
    model_seq = seq_vae.load_from_checkpoint(checkpoint_path=ckpt_file_path_seq, hparams = hparam_seq)
    # model_seq = seq_vae(**hparam_seq).load_from_checkpoint(ckpt_file_path_seq)


    # if anom==False:
    if anomaly=='norm':
        # _, data, _, _, _ = preprocess_data_simu_tank('simu_tank')
        # df_csv = pd.DataFrame(data)
        # df_csv = pd.read_csv(f'preprocessed_data/tank_simulation/norm.csv').iloc[2000:, :3].reset_index(drop=True)
        df_csv = pd.read_csv(f'preprocessed_data/tank_simulation/norm_long.csv').iloc[1500:, :3].reset_index(drop=True)
        df_csv_realcat = pd.read_csv(f'preprocessed_data/tank_simulation/norm_long.csv').iloc[1500:, 3].reset_index(drop=True)

        # df_csv_realcat = pd.read_csv(f'preprocessed_data/tank_simulation/norm.csv').reset_index(drop=True).iloc[2000:, 3].reset_index(drop=True)
        scaler = StandardScaler().fit(pd.read_csv('preprocessed_data/tank_simulation/norm_long.csv').iloc[:, :3].reset_index(drop=True))
        df_csv_sc = pd.DataFrame(scaler.transform(df_csv), columns=df_csv.columns, index=df_csv.index).iloc[:2000, :].reset_index(drop=True)
 

    else:
        df_csv = pd.read_csv(f'preprocessed_data/tank_simulation/{anomaly}_long_faulty.csv').reset_index(drop=True).iloc[:, :3]
        df_csv_realcat = pd.read_csv(f'preprocessed_data/tank_simulation/{anomaly}_long_faulty.csv').iloc[1500:2500, 3].reset_index(drop=True)
        scaler = StandardScaler().fit(pd.read_csv('preprocessed_data/tank_simulation/norm.csv').iloc[1000:, :3].reset_index(drop=True))
        df_csv_sc = pd.DataFrame(scaler.transform(df_csv), columns=df_csv.columns, index=df_csv.index).iloc[1500:2500, :].reset_index(drop=True)
  
    faulty_idx = df_csv_realcat.str.contains('faulty').astype(int)
    # df_scale = preprocess_std_data(torch.tensor(df_csv.values))
    # dataset_seq = seq_dataset(dataframe=pd.DataFrame(df_scale),  number_timesteps=hparam["hparams"]["NUMBER_TIMESTEPS"])
    # df_csv = standardize_data(df_csv, 'scaler_SWaT.pkl')
    dataset = Dataset(dataframe=df_csv_sc.iloc[:, :3].reset_index(drop=True), number_timesteps=hparam["hparams"]["NUMBER_TIMESTEPS"])


    all_cats = []
    all_kl = []
    all_like = []
    all_residuals = []
    all_mu = []
    for window in dataset:
        # Assuming 'window' is your input sequence
        predicted, x = model_seq(window.unsqueeze(0).to('cuda'))  # Replace with your prediction method
        residuals = np.abs(window - predicted.detach().cpu())  # Calculate absolute residuals
        all_residuals.extend(x['recon_loss'].flatten().detach().cpu().numpy())
        ########## SEQCAT_CATVAE
        pzx_logits, pzx, mu, sigma, pxz, z = model.get_states(window.unsqueeze(0).to('cuda'))
        _, kl = model.kl_divergence(pzx=pzx)
        # TODO: Überlegen ob mean das richtige ist
        like = model.function_likelihood(x=window.unsqueeze(0).to('cuda')).mean()
        z_list = z.detach().cpu().numpy().astype(int) 
        all_cats.append(z_list)
        all_kl.append(kl.detach().cpu().numpy())
        all_like.append(like.detach().cpu().numpy())
        all_mu.append(mu.detach().cpu().numpy())

    all_residuals = np.array(all_residuals)
    anom_labels = np.where(all_residuals < threshold, .5 , 0)

    all = pd.DataFrame(np.vstack(all_cats))
    kl_ = pd.DataFrame(np.vstack(all_kl))
    cats = pd.DataFrame(all.idxmax(axis=1))
    like_ = pd.DataFrame(all_like)
    mu_ = pd.DataFrame(np.vstack(all_mu)[::10].reshape(-1, np.vstack(all_mu)[::10].shape[2]))
    data_ = pd.DataFrame(np.vstack(dataset)[::10].reshape(-1, np.vstack(all_mu)[::10].shape[2]))
    unique_cats = cats[cats.columns[0]].unique()
    # else: 
    #     df_csv = pd.read_csv('preprocessed_data/tank_simulation/q1_faulty.csv').reset_index(drop=True).iloc[1000:, :3]
    #     df_csv_realcat = pd.read_csv('preprocessed_data/tank_simulation/q1_faulty.csv').reset_index(drop=True).iloc[1000:, 3]

    # df_sc = standardize_data(df_csv, 'scaler_SWaT.pkl')
    # df = Dataset(dataframe = df_sc)[:][0:]

    # likelihood = pd.DataFrame(model.function_likelihood(torch.tensor(df).to(device='cuda')).cpu().detach()).rolling(10).median().fillna(method='bfill')
    # pzx_logits, pzx, mu, sigma, pxz, z = model.get_states(torch.tensor(df).to(device='cuda'))
    # df_states = pd.DataFrame(torch.zeros(z.shape).to(device='cuda').scatter(1, torch.argmax(pzx_logits, dim=1).unsqueeze(1), 1).cpu().detach().numpy(), index=pd.DataFrame(df_csv).index).astype(int)
    # cats = pd.DataFrame(df_states.idxmax(axis=1), index=pd.DataFrame(df_csv).index)  

    cluster_assignments = cats[cats.columns[0]]
    class_assignments = df_csv_realcat[0:-10]
    assert len(cluster_assignments) == len(class_assignments)
    
    num_samples = len(cluster_assignments)
    num_clusters = len(np.unique(cluster_assignments))
    num_classes = len(np.unique(class_assignments))
    
    cluster_class_counts = {cluster_: {class_: 0 for class_ in np.unique(class_assignments)}
                            for cluster_ in np.unique(cluster_assignments)}
    
    for cluster_, class_ in zip(cluster_assignments, class_assignments):
        cluster_class_counts[cluster_][class_] += 1
        
    total_intersection = sum([max(list(class_dict.values())) for cluster_, class_dict in cluster_class_counts.items()])
    
    purity = total_intersection/num_samples
    print(purity)

    fig = make_subplots(rows=6, cols=1, shared_xaxes=True)
    for i in range(0,3):
        # fig.add_trace(go.Scatter(x=pd.DataFrame(df_csv).index, y=pd.DataFrame(df_csv)[pd.DataFrame(df_csv).columns[i]], name=df_csv.columns[i],  mode='markers'), 
        #               row=1, col=1)
        fig.add_trace(go.Scatter(x=pd.DataFrame(df_csv_sc).index, y=pd.DataFrame(df_csv_sc)[pd.DataFrame(df_csv_sc).columns[i]], name=df_csv_sc.columns[i],  mode='markers'), 
                    row=1, col=1)
        # fig.add_trace(go.Scatter(x=pd.DataFrame(df_csv).index, y=mu_[mu_.columns[i]], name=mu_.columns[i], mode='markers'), 
                    #   row=1, col=1)
    fig.add_trace(go.Scatter(x = pd.DataFrame(df_csv_sc).index, y=cats[cats.columns[0]], name='category', mode='lines'),row=2, col=1)
    fig.add_trace(go.Scatter(x=pd.DataFrame(df_csv_realcat).index, y=df_csv_realcat.values, name='vae cats', mode='markers'), row=3, col=1)
    fig.add_trace(go.Scatter(x=pd.DataFrame(df_csv_sc).index, y=all_residuals, name='residual', mode='markers'), row=4, col=1)
    fig.add_trace(go.Scatter(x=df_csv.index, y=like_[like_.columns[0]], mode='lines', name='likelihood'), row=5, col=1) # was row=5

    fig.add_trace(go.Scatter(x=pd.DataFrame(df_csv_sc).index, y=faulty_idx, mode='lines', name='fault'), row=6, col=1)
    fig.add_trace(go.Scatter(x=pd.DataFrame(df_csv_sc).index, y=anom_labels, mode='lines', name='anomaly'), row=6, col=1)
    print(len(cats[cats.columns[0]].unique()))

    fig.update_layout(title_text=anomaly)
    return fig
    # return cats


In [ ]:
# plot of nominal data
# anomalies = ['v12not', 'v3', 'v12', 'v23', 'norm']
# cats = plot_like(anomaly='norm', idx=1) 

interact(plot_like, idx = range(100), anomaly=['norm', 'q1not', 'v12not', 'v23not', 'v3not', 
                             'q1short1s', 'v12short1s', 'v23short1s', 'v3short1s',
                              'q1_50s', 'v12_50s', 'v23_50s', 'v3_50s',
                               'q1_100s', 'v12_100s', 'v23_100s', 'v3_100s', 
                               'rest_100s', 'q1v3_100s', 'v12v23_100s', 
                               'v12v3_100s', 'q1v23_100s', 'norm'])


In [ ]:
# computation of total purity

total_purity = []
anomaly = 'norm'
for idx in range(10):
    MODEL_VERSION = f'lightning_logs/seqcat_tank_RNN_v25/version_{idx}' # version 3 and version 25
    ckpt_file_name = os.listdir(f'./{MODEL_VERSION}/checkpoints/')[-1]
    ckpt_file_path = f'./{MODEL_VERSION}/checkpoints/{ckpt_file_name}'
    with open(f'./{MODEL_VERSION}/hparams.yaml') as f:
        hparam = yaml.safe_load(f)
    model = seqcat_vae.load_from_checkpoint(ckpt_file_path, hparams=hparam["hparams"])

    MODEL_VERSION_SEQ = f'lightning_logs/seq_vae_tank_v9/version_5' # 
    threshold = -170
    ckpt_file_name_seq = os.listdir(f'./{MODEL_VERSION_SEQ}/checkpoints/')[-1]
    ckpt_file_path_seq = f'./{MODEL_VERSION_SEQ}/checkpoints/{ckpt_file_name_seq}'
    with open(f'./{MODEL_VERSION_SEQ}/hparams.yaml') as f:
        hparam_seq = yaml.safe_load(f)
    model_seq = seq_vae.load_from_checkpoint(checkpoint_path=ckpt_file_path_seq, hparams = hparam_seq)

    df_csv = pd.read_csv(f'preprocessed_data/tank_simulation/norm_long.csv').iloc[:, :3].reset_index(drop=True)
    df_csv_realcat = pd.read_csv(f'preprocessed_data/tank_simulation/norm_long.csv').iloc[1500:, 3].reset_index(drop=True)

    # df_csv_realcat = pd.read_csv(f'preprocessed_data/tank_simulation/norm.csv').reset_index(drop=True).iloc[2000:, 3].reset_index(drop=True)
    scaler = StandardScaler().fit(pd.read_csv('preprocessed_data/tank_simulation/norm_long.csv').iloc[:, :3].reset_index(drop=True))
    df_csv_sc = pd.DataFrame(scaler.transform(df_csv), columns=df_csv.columns, index=df_csv.index).iloc[1500:, :].reset_index(drop=True)

    faulty_idx = df_csv_realcat.str.contains('faulty').astype(int)
    dataset = Dataset(dataframe=df_csv_sc.iloc[:, :3].reset_index(drop=True), number_timesteps=hparam["hparams"]["NUMBER_TIMESTEPS"])


    all_cats = []
    all_kl = []
    all_like = []
    all_residuals = []
    all_mu = []
    for window in dataset:
        # Assuming 'window' is your input sequence
        predicted, x = model_seq(window.unsqueeze(0).to('cuda'))  # Replace with your prediction method
        residuals = np.abs(window - predicted.detach().cpu())  # Calculate absolute residuals
        all_residuals.extend(x['recon_loss'].flatten().detach().cpu().numpy())
        ########## SEQCAT_CATVAE
        pzx_logits, pzx, mu, sigma, pxz, z = model.get_states(window.unsqueeze(0).to('cuda'))
        _, kl = model.kl_divergence(pzx=pzx)
        # TODO: Überlegen ob mean das richtige ist
        like = model.function_likelihood(x=window.unsqueeze(0).to('cuda')).mean()
        z_list = z.detach().cpu().numpy().astype(int) 
        all_cats.append(z_list)
        all_kl.append(kl.detach().cpu().numpy())
        all_like.append(like.detach().cpu().numpy())
        all_mu.append(mu.detach().cpu().numpy())

    #     all_residuals = np.array(all_residuals)
    # anom_labels = np.where(all_residuals < threshold, .5 , 0)

    all = pd.DataFrame(np.vstack(all_cats))
    kl_ = pd.DataFrame(np.vstack(all_kl))
    cats = pd.DataFrame(all.idxmax(axis=1))
    like_ = pd.DataFrame(all_like)
    mu_ = pd.DataFrame(np.vstack(all_mu)[::10].reshape(-1, np.vstack(all_mu)[::10].shape[2]))
    data_ = pd.DataFrame(np.vstack(dataset)[::10].reshape(-1, np.vstack(all_mu)[::10].shape[2]))
    unique_cats = cats[cats.columns[0]].unique()
    # else: 
    #     df_csv = pd.read_csv('preprocessed_data/tank_simulation/q1_faulty.csv').reset_index(drop=True).iloc[1000:, :3]
    #     df_csv_realcat = pd.read_csv('preprocessed_data/tank_simulation/q1_faulty.csv').reset_index(drop=True).iloc[1000:, 3]

    # df_sc = standardize_data(df_csv, 'scaler_SWaT.pkl')
    # df = Dataset(dataframe = df_sc)[:][0:]

    # likelihood = pd.DataFrame(model.function_likelihood(torch.tensor(df).to(device='cuda')).cpu().detach()).rolling(10).median().fillna(method='bfill')
    # pzx_logits, pzx, mu, sigma, pxz, z = model.get_states(torch.tensor(df).to(device='cuda'))
    # df_states = pd.DataFrame(torch.zeros(z.shape).to(device='cuda').scatter(1, torch.argmax(pzx_logits, dim=1).unsqueeze(1), 1).cpu().detach().numpy(), index=pd.DataFrame(df_csv).index).astype(int)
    # cats = pd.DataFrame(df_states.idxmax(axis=1), index=pd.DataFrame(df_csv).index)  

    cluster_assignments = cats[cats.columns[0]]
    class_assignments = df_csv_realcat[0:-10]
    assert len(cluster_assignments) == len(class_assignments)
    
    num_samples = len(cluster_assignments)
    num_clusters = len(np.unique(cluster_assignments))
    num_classes = len(np.unique(class_assignments))
    
    cluster_class_counts = {cluster_: {class_: 0 for class_ in np.unique(class_assignments)}
                            for cluster_ in np.unique(cluster_assignments)}
    
    for cluster_, class_ in zip(cluster_assignments, class_assignments):
        cluster_class_counts[cluster_][class_] += 1
        
    total_intersection = sum([max(list(class_dict.values())) for cluster_, class_dict in cluster_class_counts.items()])
    
    purity = total_intersection/num_samples
    print(purity)
    total_purity.append(purity)
mean = sum(total_purity)/len(total_purity)


In [ ]:
mean

In [ ]:
variance = sum([((x - mean) ** 2) for x in total_purity]) / len(total_purity) 
res = variance ** 0.5
res

In [ ]:
cats

In [ ]:
cats['Prev_Value'] = cats[cats.columns[0]].shift(1)
cats['Change'] = (cats[cats.columns[0]] != cats['Prev_Value'])
transitions = cats[cats['Change'] == True]
transitions = transitions.dropna()

In [ ]:
transitions[transitions.columns[0]].to_csv('preprocessed_data/tank_transistions.csv')

In [ ]:
transitions[transitions.columns[0]].to_list()

In [ ]:
from aalpy.learning_algs import run_Alergia
from aalpy.utils import visualize_automaton

model = run_Alergia(data=[transitions[transitions.columns[0]].to_list()], automaton_type='mc', eps=0.001, print_info=True)


In [ ]:
print(model.initial_state.output)

In [ ]:
for state in model.states:
    print(state.transitions)

In [ ]:
for state in model.states:
    print(state.transitions[0][0].transitions[0][0].transitions)


In [ ]:
for state in model.states:
    print(state.output)

In [ ]:


init_state = model.initial_state.output
for state in model.states:
    for sub in state.transitions
        print(init_state)
        print(sub[0].output)
        # print('--------------')
        print(sub[1])
        print('--------------')

    init_state = state.transitions[0][0].output



In [ ]:
# init_state = model.initial_state.output
transitions_data = []  # List to collect transition data

for state in model.states:
    for sub in state.transitions: 
        # Collect each transition data
        transition = {
            'Previous': state.output,
            'Current': sub[0].output,
            'Probability': sub[1]
        }
        transitions_data.append(transition)
        print('--------------')  # Maintain this for visual separation in console output

    # Update the initial state for the next iteration if there are any transitions
    # if state.transitions:
    # init_state = state.transitions[0][0].output

# Create DataFrame from collected data
transitions_df = pd.DataFrame(transitions_data)

In [ ]:
transitions_df.head(25)

In [ ]:
probability_distributions = [i[1] for i in model.current_state.transitions]
states = [i[0] for i in model.current_state.transitions]

In [ ]:
states

In [ ]:
# plot of anomalous data by exceeding the likelihood of -40 compared to the plot above
# plot_like(anomaly=anomalies[1])

In [ ]:
# plot_like(anomaly=anomalies[2])

In [ ]:
def define_automaton():
    automaton = {
        'A': {'B': 0.6, 'C': 0.4},
        'B': {'A': 1}
    }
    return automaton

def check_transition(current_state, next_state, automaton):
    """
    Check if the transition from current_state to next_state is allowed in the automaton.
    
    Parameters:
        current_state (str): The current state of the automaton.
        next_state (str): The proposed next state.
        automaton (dict): A dictionary representing the automaton with states and valid transitions.

    Returns:
        bool: True if the transition is valid, False otherwise.
    """
    valid_transitions = automaton.get(current_state, {})
    if next_state in valid_transitions:
        return True
    else:
        return False




In [ ]:
# Example usage
automaton = define_automaton()
current_state = 'A'
next_state = 'B'
is_valid = check_transition(current_state, next_state, automaton)
print(f"Transition from {current_state} to {next_state} is valid: {is_valid}")

current_state = 'B'
next_state = 'C'
is_valid = check_transition(current_state, next_state, automaton)
print(f"Transition from {current_state} to {next_state} is valid: {is_valid}")